# Fitting

In [1]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap, DilutedKrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss, SpectrumDistance
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic
from quantum_tools import apply_unitary, measurement, resample, state_fidelity, generate_unitary
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, mean_spacing


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function_list = [SpectrumDistance(sigma=sigma, k = None)]),
                            )
        
        model.train(inputs = None,
                    targets = [spectrum_target],
                    inputs_val = [None],
                    targets_val = [[spectrum_target]],	
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_list[0][-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 10:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## L = 8

In [7]:
n = 4
d = 2**n

model = pkl.load(open("models/L=8.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.8,
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_8layer.model")

[0.0830325839217309]
[0.06791329312758165]
[0.06857373693872561]
[0.05376585814355927]
[0.05838301400497145]
[0.04879840949151757]
[0.05795658037756955]
[0.06550335764654101]
[0.053762589767631166]
[0.05879071706207925]
Done!


In [8]:
n = 4
d = 2**n

model = pkl.load(open("models/L=16.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.8,
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_16layer.model")

[0.09116035009717097]
[0.07645605216661128]
[0.07580057879365557]
[0.06255942244020359]
[0.06852067752443668]
[0.05469897135024519]
[0.060937365867881906]
[0.059857527681661374]
[0.05386032063619552]
[0.0684080858894199]
[0.06388039206370835]
[0.05459514229296083]
[0.06183507331100109]
Done!


In [9]:
n = 4
d = 2**n

model = pkl.load(open("models/L=24.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.65,
                                                      rank_start = 4, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_24layer.model")

[0.05596892451421553]
[0.058266860342138876]
[0.05047246951151095]
[0.049618374401470246]
[0.04107484628336278]
[0.04412024287397598]
[0.04117010106987273]
[0.04092414231719024]
[0.04101289700829092]
[0.04467031224258458]
[0.03962389494293427]
[0.042153706347199366]
[0.04016756401006438]
[0.04469256040263084]
[0.04061853242517355]
Done!


In [10]:
n = 4
d = 2**n

model = pkl.load(open("models/L=32.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_32layer.model")

[0.07147791869020646]
[0.06493396168584076]
[0.06354725716818352]
[0.056586529537017746]
[0.05625653827025364]
[0.05412557584896151]
[0.05346927445553439]
[0.05072443400736197]
[0.05547207940171963]
[0.04672105334086594]
[0.053871877719061934]
[0.047848719000861926]
[0.050762750034000825]
[0.05433721571597523]
Done!


In [11]:
n = 4
d = 2**n

model = pkl.load(open("models/L=40.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.45,
                                                      rank_start = 12, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_40layer.model")

[0.05571647909861641]
[0.05856122147362428]
[0.05582918938345004]
[0.05081678749310863]
[0.048761609793911774]
[0.05314669174566853]
[0.047504799638732574]
[0.03873520195642267]
[0.04674564347046062]
[0.04427322051800795]
[0.047116804978091335]
[0.043078630341072295]
Done!


In [6]:
n = 4
d = 2**n

model = pkl.load(open("models/L=56.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.3,
                                                      rank_start = 18, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_56layer.model")

[0.05984527373286971]
[0.060827349052847875]
[0.055182651081495525]
[0.05625047431890397]
[0.0552721796932341]
[0.05587989556011875]
[0.051219878736549734]
[0.05153165099066217]
[0.04710953115472637]
[0.04617695060452495]
[0.04391109192208972]
[0.04603371100359047]
[0.047377104978761374]
[0.039426273216047955]
[0.04253131443760904]
[0.04194396677652312]
[0.036055091667030054]
[0.03866702723583215]
[0.039972487851887464]
[0.03677118707226949]
[0.03443801997422244]
[0.03928322818171888]
[0.03769943702368718]
[0.03871946002140078]
[0.034097795456278945]
[0.03518214064996711]
[0.03298110071496889]
[0.03306935164729337]
[0.03287164650935856]
[0.034298616840636634]
[0.03961358626836574]
[0.039293615230778886]
[0.035390028338908516]
Done!


In [7]:
n = 4
d = 2**n

model = pkl.load(open("models/L=72.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.2,
                                                      rank_start = 30, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_72layer.model")

[0.05034535465167999]
[0.050469782106183025]
[0.04998652957185062]
[0.04796716365526161]
[0.04732383738493544]
[0.04430162452716135]
[0.04505277549439514]
[0.043107228387178856]
[0.04495140382345537]
[0.042324757911128774]
[0.04380325546093934]
[0.03879982652195541]
[0.04252048363510318]
[0.03606330506877028]
[0.037789584689744846]
[0.034268603053727656]
[0.03525432101439853]
[0.032704541060995865]
[0.03305562801347346]
[0.03213883004262047]
[0.03287000011985777]
[0.02907095504781121]
[0.027453370807334898]
[0.03326454358340965]
[0.027882508637606787]
[0.02296935112118298]
[0.027593317936997597]
[0.02532760169611399]
[0.02364389981650024]
[0.02413049793906373]
Done!


In [5]:
n = 4
d = 2**n

model = pkl.load(open("models/L=88.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.15,
                                                      rank_start = 55, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_88layer.model")

[0.02995780224270443]
[0.03214226838020823]
[0.02677864965119864]
[0.023108335862412024]
[0.025764057851014543]
[0.025126394009863745]
[0.026362204282625937]
[0.0245049265346778]
[0.025119063536654107]
[0.026075681358102037]
[0.026581193455600893]
[0.025038721781134074]
[0.023803146933332686]
[0.02269965835378182]
[0.023877246588756243]
[0.024148722962237495]
[0.0239788503501277]
[0.01903591247308524]
[0.020914753103185926]
[0.024317400229313446]
[0.021766773960891966]
[0.02261239115772916]
[0.02241036805277974]
[0.021469916298406194]
[0.020604249547063358]
[0.02095227556168093]
[0.019904322819012277]
[0.01878016332759705]
[0.02084036315329177]
[0.02187285605881424]
[0.019836582291889712]
[0.020933829556486053]
[0.02539048842701845]
[0.020350097924756547]
[0.025847381850247278]
[0.022468873557302066]
[0.024212639870921024]
[0.024444675598754864]
[0.023696123734244443]
Done!


In [6]:
n = 4
d = 2**n

model = pkl.load(open("models/L=104.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.12,
                                                      rank_start = 60, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_104layer.model")

[0.033269510496164154]
[0.028009451676414776]
[0.026792426451400217]
[0.025197741808709068]
[0.029575579038326555]
[0.029655339591170504]
[0.02618156661281526]
[0.02590710060823373]
[0.02741172565336565]
[0.02415402420745072]
[0.02563458828522558]
[0.02176140091674658]
[0.022886311661030848]
[0.02395630240799189]
[0.02222053704991145]
[0.022052768849921717]
[0.021517224857834485]
[0.023641513301921366]
[0.022275025150951755]
[0.019489233438316392]
[0.021673134370369565]
[0.023157041135813197]
[0.02100324017656474]
[0.023537355178232314]
[0.021003459604099398]
[0.021407576523849073]
[0.021879148442064317]
[0.019734814709043424]
[0.02032532286365256]
[0.022578090112218924]
[0.021958518494120574]
Done!


In [7]:
n = 4
d = 2**n

model = pkl.load(open("models/L=120.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.07,
                                                      rank_start = 60, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_120layer.model")

[0.023346902732596986]
[0.021782160184933938]
[0.01932639330822121]
[0.022209133469524953]
[0.020603647364506525]
[0.01857969448680763]
[0.020834054071106944]
[0.01857003431565646]
[0.019548934278783583]
[0.020483236275594868]
[0.020771703470935196]
[0.01989474698786363]
[0.019261421247203536]
[0.01722148369414156]
[0.019368571340822344]
[0.02320680293250295]
[0.019365672645055718]
[0.020702580304016255]
[0.019941254977479728]
[0.017918676774303186]
[0.018420815279027544]
[0.018724170473128784]
[0.019259068203065277]
[0.019936043905844553]
[0.019256057638476695]
Done!
